In [1]:
#import libraries to use
import pandas as pd
import numpy as np

In [29]:
#import the respective data as a dataframe
data = pd.read_csv('buyers_RAW.csv',low_memory = False)

In [30]:
#check out the first 5 rows of data, head defaults to 5, you can also do buyer_data.head(10) to show 10 rows, etc.
data.head()

,month,year,user_id,currency,count,revenue,Date,USD,Per Purchase
0,6,2015,1154979,GBP,2,37.29,20156,55.94,27.97
1,10,2015,2677378,GBP,1,17,201510,25.50,25.5
2,9,2015,546489,GBP,3,118.5,20159,177.76,59.25
3,8,2015,846185,GBP,4,66.79,20158,100.19,25.05
4,6,2015,2099822,GBP,2,29.98,20156,44.97,22.49


In [31]:
#group the data by user_id and then take the minimum of month to get their cohort,
#as_index = False indicates that you don't want the user_id to be the index of the result
#it should be a column of its own
cohort = data.groupby('user_id',as_index = False)['month'].min()

In [32]:
#change the cohort object to a dataframe so it can be merged with buyer_data
cohort = pd.DataFrame(cohort)

In [33]:
#merge the two dataframes, left_on and right_on indicates which column to match the two
df = pd.merge(data,cohort,left_on = 'user_id', right_on = 'user_id')

In [34]:
#reset the name of the columns for clarity, when you merge, the columns are just named month_x and 
#month_y instead of month and cohort
df.columns = ['month','year','user_id','currency','count','revenue','Date','USD',
               'Per Purchase','cohort']

In [35]:
#check to make sure merge was successful
df.head()

,month,year,user_id,currency,count,revenue,Date,USD,Per Purchase,cohort
0,6,2015,1154979,GBP,2,37.29,20156,55.94,27.97,5
1,5,2015,1154979,GBP,2,10,20155,15.00,7.5,5
2,10,2015,2677378,GBP,1,17,201510,25.50,25.5,10
3,9,2015,546489,GBP,3,118.5,20159,177.76,59.25,8
4,8,2015,546489,GBP,1,17.8,20158,26.70,26.7,8


In [36]:
#to get monthly revenue by cohort, group by cohort and month, then sum the USD column
cohort_revenue = df.groupby(['cohort','month'],as_index = False)['USD'].sum()

In [37]:
#sort the value by cohort and month if you want, cohort 1 should have month 1-10, 2 should have month 2-10...etc.
cohort_revenue = cohort_revenue.sort_values(by=['cohort','month'])

In [38]:
#check the result
cohort_revenue.head()

,cohort,month,USD
0,1,1,1166403.69
1,1,2,471882.58
2,1,3,436972.20
3,1,4,382938.63
4,1,5,387836.70


In [39]:
#get monthly uniques by cohort, group by cohort and month, then add up the unique users
cohort_uniques = df.groupby(['cohort','month'],as_index = False).apply(lambda row: row.user_id.nunique())

In [40]:
#check out the first few rows
cohort_uniques.head()

cohort  month
1       1        26381
        2         9639
        3         8950
        4         7955
        5         7846
dtype: int64

In [41]:
#save analysis to csv
cohort_uniques = pd.DataFrame(cohort_uniques)
cohort_uniques.columns = ['uniques']
cohort_uniques.to_csv('buyers_uniques.csv')
cohort_revenue.to_csv('buyers_revenue.csv')